### Step 1: Install CANYA

Google Colab 환경에서 Conda 없이 바로 CANYA를 설치합니다. 필요한 종속성(`tensorflow`, `numpy`, `pandas`)은 Colab에 이미 설치되어 있으므로 별도 설치가 필요하지 않습니다.

In [1]:
# --- 1단계: Python 3.9 설정 및 빌드 환경 구축 ---

# 파이썬 3.9 설치 및 기본값 설정
!sudo apt-get update -y
!sudo apt-get install python3.9 python3.9-distutils

# ❗ C/C++ 컴파일러 등 필수 빌드 도구 설치 (numpy, pandas 빌드 오류 해결의 핵심)
!sudo apt-get install -y build-essential

# 파이썬 기본값 변경 및 pip 설치
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.9 1
!curl -sS https://bootstrap.pypa.io/get-pip.py | python3.9

# --- 2단계: setup.py 기반 의존성 패키지 설치 ---

# setup.py에 명시된 정확한 버전들을 설치합니다.
# 빌드 도구가 설치되었으므로 이제 numpy와 pandas가 정상적으로 컴파일됩니다.
!python3 -m pip install numpy==1.19.5
!python3 -m pip install pandas==1.3.3
!python3 -m pip install tensorflow==2.6.0
!python3 -m pip install keras==2.6.0
!python3 -m pip install "protobuf<=3.20" # setup.py의 protobuf<=3.20 요구사항 충족

# --- 3단계: CANYA 설치 ---

# 이제 모든 의존성이 해결되었으므로 canya를 설치합니다.
!python3 -m pip install --no-cache-dir https://github.com/lehner-lab/canya/tarball/master

print("\n\n✅ 모든 설치가 성공적으로 완료되었습니다. 이제 다음 셀부터 CANYA 코드를 실행할 수 있습니다.")

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,758 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [4,963 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.com/ubuntu jammy-security/universe amd64

### Step 2: Prepare Input File

`canya_input.txt`라는 입력 파일을 생성합니다. 형식은 CANYA에서 요구하는 형식에 맞게 구성해야 합니다 (예: 탭 구분 서열 리스트).

In [3]:
with open("canya_input.txt", "w") as f:
    f.write("ID\tSequence\n")
    f.write("Seq1\tCASSIRSSYEQYF\n")
    f.write("Seq2\tCASSLGQGAETLYF\n")

### Step 3: Run CANYA (Basic Mode)

In [4]:
!canya --input canya_input.txt --output basic_out.txt

/bin/bash: line 1: canya: command not found


### Step 4: Run CANYA with Summary Mode (`--summarize mean`)

In [5]:
!canya --input canya_input.txt --output summarized_out.txt --summarize mean

/bin/bash: line 1: canya: command not found


### Step 5: Run CANYA in Ensemble Mode (Robust Prediction)

In [6]:
!canya --input canya_input.txt --output ensemble_out.txt --summarize mean --mode ensemble

/bin/bash: line 1: canya: command not found


### Step 6: Visualize Results

In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

try:
    df = pd.read_csv("ensemble_out.txt", sep='\t')
    df_sorted = df.sort_values('Mean_Score', ascending=False).reset_index(drop=True)

    plt.figure(figsize=(10, 6))
    sns.barplot(x='ID', y='Mean_Score', data=df_sorted, palette='viridis', edgecolor='black')
    plt.errorbar(x=df_sorted.index, y=df_sorted['Mean_Score'], yerr=df_sorted['Std_Dev_Score'], fmt='none', c='black', capsize=5)
    plt.title('CANYA Nucleation Score (Ensemble Mode)', fontsize=16)
    plt.xlabel('Sequence ID')
    plt.ylabel('Mean ± Std Dev')
    plt.ylim(0, 1)
    plt.tight_layout()
    plt.show()
except Exception as e:
    print("Error reading results:", e)

Error reading results: [Errno 2] No such file or directory: 'ensemble_out.txt'
